In [2]:
import instaloader
from datetime import datetime, timedelta
import csv

L = instaloader.Instaloader()

# The usernames of the political parties on Instagram
party_usernames = ["partido_socialista", "partidosocialdemocrata", "partidochega", "liberalpt", "blocodeesquerdaoficial", "pcp.pt", "partidolivre", "partido_pan"]

# Define the start and end date of the time period
start_date = datetime(2024, 1, 6)
end_date = datetime(2024, 3, 9) 

# Save it into a CSV file
with open('frequency_of_the_instagram_posts.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Political Party', 'Username', 'Post Date']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    # Iterate through each party and retrived their posts made in the time period
    for party in party_usernames:
        print(f"Fetching posts for {party}")
        try:
            profile = instaloader.Profile.from_username(L.context, party)

            # Download those posts
            posts = profile.get_posts()
            for post in posts:
                if start_date <= post.date_utc <= end_date:
                    writer.writerow({'Political Party': party, 'Username': profile.username, 'Post Date': post.date_utc.strftime("%Y-%m-%d %H:%M:%S")})
                elif post.date_utc < start_date:
                    break  # Stop fetching posts if the post date is before the start date
        except instaloader.exceptions.InstaloaderException as e:
            print(f"Error fetching posts for {party}: {e}")  # this is to see if there were any errors getting the data 
        except Exception as e:                                        # for some party
            print(f"An unexpected error occurred for {party}: {e}")

print("Data saved to frequency_of_the_instagram_posts.csv")

Fetching posts for partido_socialista
Fetching posts for partidosocialdemocrata
Fetching posts for partidochega
Fetching posts for liberalpt
Fetching posts for blocodeesquerdaoficial
Fetching posts for pcp.pt
Fetching posts for partidolivre
Fetching posts for partido_pan
Data saved to frequency_of_the_instagram_posts.csv


In [5]:
import pandas as pd

df = pd.read_csv('frequency_of_the_instagram_posts.csv')

# Define the names for each party
party_mapping = {
    'partido_socialista': 'PS',
    'partidosocialdemocrata': 'PPD/PSD.CDS-PP.PPM',
    'partidochega': 'CH',
    'liberalpt': 'IL',
    'blocodeesquerdaoficial': 'B.E.',
    'pcp.pt': 'PCP',
    'partidolivre': 'L',
    'partido_pan': 'PAN'
}

# Replace old party names with new ones
df['Political Party'] = df['Political Party'].replace(party_mapping)

# Save it to CSV
df.to_csv('frequency_of_the_instagram_posts.csv', index=False)

In [6]:
import pandas as pd

posts_df = pd.read_csv("frequency_of_the_instagram_posts.csv")

# Convert 'Post Date' to datetime
posts_df['Post Date'] = pd.to_datetime(posts_df['Post Date'])

# Group data by 'Party' and the weekly period of 'Post Date', then count the number of posts
party_weekly_post_frequency = posts_df.groupby(['Political Party', pd.Grouper(key='Post Date', freq='W-SUN')]).size()

# Reset index to make it a dataframe
party_weekly_post_frequency = party_weekly_post_frequency.reset_index()

# Rename the columns
party_weekly_post_frequency.columns = ['Political Party', 'Week_Start', 'Frequency']

party_weekly_post_frequency

,Political Party,Week_Start,Frequency
0,B.E.,2024-01-07,2
1,B.E.,2024-01-14,9
2,B.E.,2024-01-21,9
3,B.E.,2024-01-28,12
4,B.E.,2024-02-04,11
...,...,...,...
72,PS,2024-02-11,31
73,PS,2024-02-18,32
74,PS,2024-02-25,30
75,PS,2024-03-03,45
